In [ ]:
## For data handling
import pandas as pd
import numpy as np

## For plotting
import matplotlib.pyplot as plt
import seaborn as sns

## This sets the plot style
## to have a grid on a white background
sns.set_style("whitegrid")

# Categorical Data In `sklearn` Tree Based Algorithms

The notes in this notebook are adapted from this blog post <a href="https://roamanalytics.com/2016/10/28/are-categorical-variables-getting-lost-in-your-random-forests/">https://roamanalytics.com/2016/10/28/are-categorical-variables-getting-lost-in-your-random-forests/</a>.

Before moving on to the next algorithm we're going to take a quick aside and discuss a shortcoming in `sklearn`'s tree based algorithms. Check out the following screenshot from the `sklearn` docs page
<img src="sklearn_tree_int.png"></img>

So what's the problem here? `DecisionTreeClassifier.fit(X,y)` takes our features, $X$, and casts them as floats which are decimals. This is fine for continuous predictors, binary predictors and probably ordinals. But, what about true categorical variables? Probably not for the best.

However, we should be okay right? We have one-hot encoding! Even this isn't ideal. Suppose you have one categorical predictor with ten categories, that's nine additional variables. What about one with $100$ variables, or more? This quickly becomes a problem with more complicated categorical variables. As we'll see below with a cooked up example.

There are two main problems with one hot encoding and tree based methods:
<ol>
    <li> 
        When you one-hot encode a variable with many categories you end up with many sparse columns. The resulting sparsity virtually ensures that continuous variables are assigned higher feature importance.
    </li>
    <li>
        A single level of a categorical variable must meet a very high bar in order to be selected for splitting early in the tree building. This can degrade predictive performance.
    </li>
</ol>

Let's examine this with our phony data.

We'll use Nick Dingwall, Chris Potts's code to generate the data but here's the idea.

Take a categorical variable, $c$, that takes values from two sets $C^+$ and $C^-$ each with $100$ unique categories, and a variable $z$ that is continuous. The target value $y$ has the following true classification rule:
$$
y = \left\lbrace \begin{array}{c c}
    1, & \text{ if } & c\in C^+ \text{ or } z > 10\\
    0, & \text{ else.}
\end{array}
\right.
$$

Just like in the real world when you collect the data you don't know this real classification rule, so you collect $10,000$ samples with $y$, $z$, and $c$ values as well as $100$ additional potential predictors (all of which are continuous for simplicity).

In [ ]:
# We import Dingwall and Potts's python code to generate the data
from tree_categorical_variables import *

In [ ]:
# data_categorical is the collected data set
# data_onehot is the onehot encoded version
data_categorical, data_onehot = generate_dataset(
    num_x=100, n_samples=10000, n_levels=200)

In [ ]:
# Examine the data as it was collected
data_categorical.head()

In [ ]:
# Examine the onehot encoded data
data_onehot.head()

Now build a decision tree that goes three layers deep and see what happens. Don't worry about the train test split here. Mess around with adding more layers if you'd like as well.

In [ ]:
# Code Here
# Sample Solution
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree

In [ ]:
# Code Here
# Sample Solution
tree_clf = DecisionTreeClassifier(max_depth = 3)
tree_clf.fit(data_onehot.iloc[:,1:],data_onehot.iloc[:,0])

plt.figure(figsize = (15,40))

plot_tree(tree_clf, filled = True, feature_names = data_onehot.columns[1:])
plt.show()

### Dear Diary 

Write down what you observed here. Is the decision tree able to identify the importance of the categorical variable?









Now play around with `sklearn`'s random forest classifier. Build a classifier and then look at the feature importance of the variables. What do you see?

In [ ]:
# Code Here
# Sample Solution
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# Code Here
# Sample Solution
forest_clf = RandomForestClassifier(n_estimators = 100, max_depth = 3)
forest_clf.fit(data_onehot.iloc[:,1:],data_onehot.iloc[:,0])

In [ ]:
# Code Here
# Sample Solution
names = []
scores = []
for name, score in zip(data_onehot.columns[1:],forest_clf.feature_importances_):
    names.append(name)
    scores.append(np.round(score,4))
    
score_df = pd.DataFrame({'feature':names,'importance_score':scores})

score_df.sort_values('importance_score',ascending=False).reset_index(drop=True).head(50)

### Dear Diary 2

Write down what you observed about the random forest classifier here










## What to Do?

So we can see that `sklearn`'s tree based methods can struggle with categorical variables. And let's point out that the phony data set we created isn't too unrealistic. Imagine you're trying to predict how a state will vote in an upcoming election. Let's say you're trying to predict if a county will vote Democrat. $c$ could be the state the county is in. We could take $C^+$ to be the states that have historically voted Democrat and $C^-$ those that either flip or historically vote Republican. Say $z$ is the latest polling data.

It's not unreasonable that you may encounter problems like this in your career as a data scientist. There are a couple of approaches you could take to address this shortcoming.
<ul>
   <li> If you're not in a production environment, like a personal project or research project, you may want to try R. R considers all categories of $c$ at once in its `rpart` package. However, R's `randomForest` package may have limitations on the number of unique categories $c$ has.</li>
   <li> There are python packages that address this shortcoming. One example is <a href = "http://docs.h2o.ai/h2o/latest-stable/h2o-py/docs/intro.html">h2o</a>, but this package requires a bit more technical work than we'll go into during this course. However, you are encouraged to explore it on your own.</li>
</ul>

The important take away is that you should be careful using `sklearn` building decision trees on data with high cardinality categorical variables.